In [14]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Step 1: Load Metadata
meta_data = pd.read_csv("metadata.csv")

# Step 2: Identify participants
# Participants who only know English
only_english = meta_data[meta_data[['spanish', 'german', 'french']].sum(axis=1) == 0]['participant'].tolist()
print(only_english)
# Participants who know English and Spanish
bilingual_spanish = meta_data[meta_data['spanish'] == 1]['participant'].tolist()
print(bilingual_spanish)
'''
# Step 2: Load ERP Data
path = "EEG_Measurements"  # Update this path to where your ERP files are located
data = []

# Define the N400 time window (300 ms to 500 ms)
n400_start_time = 300  # Starting time point in milliseconds
n400_end_time = 500    # Ending time point in milliseconds

# Calculate row indices for the N400 time window
n400_start_index = n400_start_time + 100  # +100 because time starts at -100 ms
n400_end_index = n400_end_time + 100

# Iterate through the files in the specified directory
for file in os.listdir(path):
    if "spanish-english" in file or "english-english" in file:
        # Load the ERP data
        df = pd.read_csv(os.path.join(path, file))

        # Get the participant ID from the filename (assuming it follows the convention)
        participant_id = file.split('_')[-1].split('.')[0]  # Extract participant number

        # Add the label based on the file name
        label = 1 if "spanish-english" in file else 0  # 1 for bilingual, 0 for monolingual
        
        # Extract the N400 window (rows from n400_start_index to n400_end_index)
        n400_window = df.iloc[n400_start_index:n400_end_index]

        # Calculate features: mean, peak, and variance within the N400 window
        n400_mean = n400_window.mean(axis=0)
        n400_peak = n400_window.max(axis=0)
        n400_variance = n400_window.var(axis=0)

        # Create a DataFrame for the calculated features
        feature_df = pd.DataFrame({
            'n400_mean': n400_mean,
            'n400_peak': n400_peak,
            'n400_variance': n400_variance
        }).T  # Transpose to have features as rows

        # Append participant ID and label
        feature_df['label'] = label
        feature_df['participant'] = participant_id

        # Append the processed data
        data.append(feature_df)

# Combine all data into a single DataFrame
all_data = pd.concat(data, ignore_index=True)
print("Combined Data Shape:", all_data.shape)
print(all_data.head())  # Display the first few rows of the combined data

# Step 3: Separate Features and Labels
X = all_data[['n400_mean', 'n400_peak', 'n400_variance']]
y = all_data['label']

# Step 4: Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Check shapes and data types
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_train dtypes:", X_train.dtypes)
print("y_train dtypes:", y_train.dtypes)

# Check for NaN values
print("NaN values in X_train:", X_train.isnull().sum())
print("NaN values in y_train:", y_train.isnull().sum()) 

# Remove rows with NaN values if necessary
X_train = X_train.dropna()
y_train = y_train[X_train.index]  # Ensure y_train matches X_train after dropping NaNs

# Step 5: Train Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Step 6: Evaluate Model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Step 7: Feature Importances
feature_importances = model.feature_importances_
print("Feature Importances:", feature_importances)
'''

[1, 2, 3, 5, 6, 7, 8, 9, 11, 20, 24, 25, 27, 29, 30, 32, 34, 36, 37, 38, 40]
[10, 13, 14, 15, 16, 17, 19, 31, 35, 39]


'\n# Step 2: Load ERP Data\npath = "EEG_Measurements"  # Update this path to where your ERP files are located\ndata = []\n\n# Define the N400 time window (300 ms to 500 ms)\nn400_start_time = 300  # Starting time point in milliseconds\nn400_end_time = 500    # Ending time point in milliseconds\n\n# Calculate row indices for the N400 time window\nn400_start_index = n400_start_time + 100  # +100 because time starts at -100 ms\nn400_end_index = n400_end_time + 100\n\n# Iterate through the files in the specified directory\nfor file in os.listdir(path):\n    if "spanish-english" in file or "english-english" in file:\n        # Load the ERP data\n        df = pd.read_csv(os.path.join(path, file))\n\n        # Get the participant ID from the filename (assuming it follows the convention)\n        participant_id = file.split(\'_\')[-1].split(\'.\')[0]  # Extract participant number\n\n        # Add the label based on the file name\n        label = 1 if "spanish-english" in file else 0  # 1 for b

In [16]:

# Step 2: Identify participants
only_english_ids = meta_data[meta_data[['Spanish', 'German', 'French']].sum(axis=1) == 0]['participant'].tolist()
bilingual_spanish_ids = meta_data[meta_data['Spanish'] == 1]['participant'].tolist()

# Step 3: Load ERP Data
path = "EEG_Measurements"  # Update this path to where your ERP files are located
data_english_english = []  # List to store English-English data
data_spanish_english = []  # List to store Spanish-English data

# Iterate through the files in the specified directory
for file in os.listdir(path):
    if file.endswith('.csv'):
        # Load the ERP data
        df = pd.read_csv(os.path.join(path, file))

        # Get the participant ID from the filename
        participant_id = file.split('_')[-1].split('.')[0]  # Extract participant number
        
        if int(participant_id) in only_english_ids:
            if "book_english-english" in file:
                data_english_english.append(df)
            elif "book_spanish-english" in file:
                data_spanish_english.append(df)
        elif int(participant_id) in bilingual_spanish_ids:
            if "book_english-english" in file:
                data_english_english.append(df)
            elif "book_spanish-english" in file:
                data_spanish_english.append(df)

# Step 4: Calculate Averages for Each Condition
# Assuming the time window is relevant for the word "book"
# Adjust the electrode index range if necessary
n400_window_start = 300  # Starting index for N400 window
n400_window_end = 500    # Ending index for N400 window

# Create lists to hold the average values for the two conditions
averages_english = []
averages_spanish = []

# Calculate average electrode values for English-English condition
for df in data_english_english:
    # Calculate mean across the N400 window
    mean_values = df.iloc[:, n400_window_start:n400_window_end].mean(axis=1)
    averages_english.append(mean_values.mean())  # Append the average for this participant

# Calculate average electrode values for Spanish-English condition
for df in data_spanish_english:
    # Calculate mean across the N400 window
    mean_values = df.iloc[:, n400_window_start:n400_window_end].mean(axis=1)
    averages_spanish.append(mean_values.mean())  # Append the average for this participant

# Step 5: Create the Final DataFrame
# Create a DataFrame with participants as columns
participants = only_english_ids + bilingual_spanish_ids  # Combine the two participant lists
average_df = pd.DataFrame(columns=participants, index=["English-English", "Spanish-English"])

# Fill the DataFrame with average values
average_df.loc["English-English"] = averages_english + [None] * (len(participants) - len(averages_english))
average_df.loc["Spanish-English"] = averages_spanish + [None] * (len(participants) - len(averages_spanish))

# Display the final DataFrame
print(average_df)

KeyError: "None of [Index(['Spanish', 'German', 'French'], dtype='object')] are in the [columns]"